In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer
import dlib
from imutils import face_utils
import time
import face_recognition

2023-03-31 15:15:30.536177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 15:15:31.581418: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-03-31 15:15:31.581509: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-31 15:15:31.719490: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already be

pygame 2.1.2 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
cd "/home/user/Downloads/DDDDeep/EY_Dataset/dataset_new"    

/home/user/Downloads/DDDDeep/EY_Dataset/dataset_new


In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier('/home/user/Downloads/DDDDeep/Mouth.xml')
model = load_model('Eye_dataset/models/model_eyes1.h5')
model2 = load_model('yawn_dataset/models/model_yawn1.h5')


2023-03-31 15:16:56.223744: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-31 15:16:56.224011: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (user-HP-Laptop-15-bs0xx): /proc/driver/nvidia/version does not exist
2023-03-31 15:16:56.230418: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
#only eye

from pygame import mixer

mixer.init()
sound= mixer.Sound(r'/home/user/Downloads/DDDDeep/alarm.wav')
cap = cv2.VideoCapture(0)

Score = 0
yawn_thresh = 10
yawn_count = 0



# initialize face detection and landmark prediction models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/home/user/Downloads/DDDDeep/shape_predictor_68_face_landmarks.dat")

# load face recognition model and database of known faces
known_face_encodings = []
known_face_names = []
for i in range(3):
    img = face_recognition.load_image_file('/home/user/Downloads/Aswanth_A.jpeg')
    face_encoding = face_recognition.face_encodings(img)[0]
    known_face_encodings.append(face_encoding)
    known_face_names.append(f'Aswanth {i}')




while True:
    ret, frame = cap.read()
    height,width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces= face_cascade.detectMultiScale(gray, scaleFactor= 1.2, minNeighbors=3)
    eyes= eye_cascade.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=1)
    mouths = mouth_cascade.detectMultiScale(gray, scaleFactor= 1.1, minNeighbors=3, minSize=(30, 30))


    cv2.rectangle(frame, (0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h), color= (145,0,0), thickness=3 )

 # loop over each face
    for face in faces:
         # extract facial landmarks and convert them to NumPy array
        shape = predictor(gray, face)
        shape = face_utils.shape_to_np(shape)
        
        # extract the face encoding
        encoding = face_recognition.face_encodings(frame, [face])[0]
        
        # compare the face encoding with known faces
        matches = face_recognition.compare_faces(known_face_encodings, encoding)
        name = "Unknown"
        for i in range(len(matches)):
            if matches[i]:
                name = known_face_names[i]
                break


      # detect yawns
    for (mx, my, mw, mh) in mouths:
        mouth = frame[my:my+mh, mx:mx+mw]
        mouth = cv2.resize(mouth, (145, 145))
        mouth = mouth / 255.0
        mouth = np.reshape(mouth, (145, 145, 3))
        mouth = np.expand_dims(mouth, axis=0)

        
        if my > height/2:
            cv2.rectangle(frame, pt1=(mx, my), pt2=(mx+mw, my+mh), color=(0, 255, 0), thickness=3)
            yawn_count += 1
        else:
            yawn_count = 0




        if yawn_count > yawn_thresh:
            cv2.putText(frame, 'yawn', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                       thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score'+str(Score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                       thickness=1, lineType=cv2.LINE_AA)
            Score += 1
            if Score > 15:
                try:
                    sound.play()
                except:
                    pass
            yawn_count = 0


    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(frame,pt1=(ex,ey),pt2=(ex+ew,ey+eh), color= (255,0,0), thickness=3 )
        
        # preprocessing steps
        eye= frame[ey:ey+eh,ex:ex+ew]
        eye= cv2.resize(eye,(145,145))
        eye= eye/255
        eye= eye.reshape(145,145,3)
        eye= np.expand_dims(eye,axis=0)
        # preprocessing is done now model prediction
        prediction = model.predict(eye)
        
        # if eyes are closed
        if prediction[0][0]>0.30:
            cv2.putText(frame,'closed',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            cv2.putText(frame,'Score'+str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            Score=Score+1
            if(Score>6):
                try:
                    sound.play()
                except:
                    pass
            
        # if eyes are open
        elif prediction[0][1]>0.90:
            cv2.putText(frame,'open',(10,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)      
            cv2.putText(frame,'Score'+str(Score),(100,height-20),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=1,color=(255,255,255),
                       thickness=1,lineType=cv2.LINE_AA)
            Score = Score-1
            if (Score<0):
                Score=0
            
        
    cv2.imshow('frame',frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[ WARN:0@224.821] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


AttributeError: 'NoneType' object has no attribute 'shape'

In [6]:

mixer.init()
sound = mixer.Sound(r'/home/user/Downloads/DDDDeep/alarm.wav')
cap = cv2.VideoCapture(0)
Score = 0
yawn_count = 0
yawn_thresh = 10

while True:
    ret, frame = cap.read()
    height, width = frame.shape[0:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=1)
    mouths = mouth_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    cv2.rectangle(frame, (0, height-50), (200, height), (0, 0, 0), thickness=cv2.FILLED)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, pt1=(x, y), pt2=(x+w, y+h), color=(145, 0, 0), thickness=3)

    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(frame, pt1=(ex, ey), pt2=(ex+ew, ey+eh), color=(255, 0, 0), thickness=3)

        # preprocessing steps
        eye = frame[ey:ey+eh, ex:ex+ew]
        eye = cv2.resize(eye, (145, 145))
        eye = eye/255
        eye = eye.reshape(145, 145, 3)
        eye = np.expand_dims(eye, axis=0)
        # preprocessing is done now model prediction
        prediction = model.predict(eye)

        # if eyes are closed
        if prediction[0][0] > 0.30:
            cv2.putText(frame, 'closed', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                       thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score'+str(Score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                       thickness=1, lineType=cv2.LINE_AA)
            Score += 1
            if Score > 10:
                try:
                    sound.play()
                except:
                    pass

        # if eyes are open
        elif prediction[0][1] > 0.70:
            cv2.putText(frame, 'open', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                       thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score'+str(Score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                       thickness=1, lineType=cv2.LINE_AA)
            Score = Score-1
            if Score < 0:
                Score = 0

 # detect yawns
    for (mx, my, mw, mh) in mouths:
        mouth = frame[my:my+mh, mx:mx+mw]
        mouth = cv2.resize(mouth, (145, 145))
        mouth = mouth / 255.0
        mouth = np.reshape(mouth, (145, 145, 3))
        mouth = np.expand_dims(mouth, axis=0)
        mouth = np.expand_dims(mouth, axis=-1)

# predict yawn probability with pre-trained model
        proba = model2.predict(mouth)[0][0]

                # update yawn count and score
        if proba > 0.5:
            yawn_count += 1
            if yawn_count > yawn_thresh:
                cv2.putText(frame, 'yawn', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                           thickness=1, lineType=cv2.LINE_AA)
                cv2.putText(frame, 'Score'+str(Score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                           thickness=1, lineType=cv2.LINE_AA)
                Score += 1
                if Score > 15:
                    try:
                        sound.play()
                    except:
                        pass
                yawn_count = 0
        else:
            yawn_count = 0

    # display frame with scores and detections
    cv2.rectangle(frame, pt1=(0,height-50), pt2=(200,height), color=(0,0,0), thickness=cv2.FILLED)
    cv2.putText(frame, 'Score'+str(Score), (100,height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
               thickness=1, lineType=cv2.LINE_AA)

    cv2.imshow('frame',frame)
    if cv2.waitKey(33) & 0xFF==ord('q'):
                break
            
    cap.release()
    cv2.destroyAllWindows()

'''
        if my > height/2:
            cv2.rectangle(frame, pt1=(mx, my), pt2=(mx+mw, my+mh), color=(0, 255, 0), thickness=3)
            yawn_count += 1
        else:
            yawn_count = 0


        if yawn_count > yawn_thresh:
            cv2.putText(frame, 'yawn', (10, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                       thickness=1, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score'+str(Score), (100, height-20), fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, fontScale=1, color=(255, 255, 255),
                       thickness=1, lineType=cv2.LINE_AA)
            Score += 1
            if Score > 6:
                try:
                    sound.play()
                except:
                    pass
            yawn_count = 0
     '''


[ WARN:0@64.310] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


AttributeError: 'NoneType' object has no attribute 'shape'